In [17]:
#importing the necessary libraries
import pandas as pd
import nltk
import matplotlib as plt
import sklearn
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn import naive_bayes
from sklearn.metrics import roc_auc_score,accuracy_score

In [2]:
#reading the dataset
df=pd.read_csv(r'C:\Users\Swarupa V\Desktop\IT-ticket classification\bank_complaint.csv')

In [3]:
df.head()

,ticket,subclass,class,class id
0,I had recently received a check for a settleme...,Problems caused by my funds being low,bank_account,1
1,"At Bank Of America, I have had my personal & b...",Problems caused by my funds being low,bank_account,1
2,"I Bank with Wells fargo, I went in to deposit ...",Problems caused by my funds being low,bank_account,1
3,Charges for my purchases were shown pending on...,Problems caused by my funds being low,bank_account,1
4,I made purchases after my direct deposit was m...,Problems caused by my funds being low,bank_account,1


In [4]:
#finding null values
df.isnull().sum()

ticket      0
subclass    0
class       0
class id    0
dtype: int64

In [5]:
x=list(df['ticket'])

In [18]:
#we need to remove numbers,symbols as they don't help in determining class, convert all to lower case, remove stop words, and lemmatize the word
stop_words = set(stopwords.words('english'))
ps = PorterStemmer()
def preprocess(s):
    lst=[]
    s=' '.join(s.split()) #removes extra white spaces+ \n,\t
    s=re.sub(r'[^\w]', ' ', s) #removes symbols
    s=s.lower()#to lower case
    s=[ps.stem(x) for x in word_tokenize(s)]#stem words and tokentise it
    s=[x for x in s if not x in stop_words] #remove stop words
    
    return s

In [35]:
#calling preprocess
X=[]
for s in x:
    s=preprocess(s)
    X.append(' '.join(s))

In [36]:
X[157]

'appear rebil payment made prior statement wa paid full amount question 330 00 balanc due 2015 statement paid full assum payment credit new charg amount correct current statement wa state amount previou period payment wa made thu instead new balanc due 650 00 would 310 00 howev appli standard account practic calcul charg less ani credit identifi balanc charg credit statement without deal ani prior payment made payment full pay time sub account get follow audit charg statement agre 2100 00 less credit shown statement incl ani previou payment 1900 00 balanc due 240 00 tri reconcil thi issu written phone commun howev'

In [50]:
#fitting the preprocessed data through a count vectoriser to represent each word as a vector of its frequency
cv = CountVectorizer(binary=True)
cv.fit(X)
x_final=cv.transform(X)

In [51]:
y=df['class']

In [89]:
X_train, X_test, y_train, y_test = train_test_split(x_final, y, train_size = 0.75,random_state=14)

In [91]:
#Multinomial Naive Bayes
clf = naive_bayes.MultinomialNB()
clf.fit(X_train, y_train)
print ("Final Accuracy: %s" % accuracy_score(y_test, clf.predict(X_test)))

Final Accuracy: 0.81


In [92]:
#multi class logistic regression
final_model = LogisticRegression(multi_class='multinomial')
final_model.fit(X_train, y_train)
print ("Final Accuracy: %s"  % accuracy_score(y_test, final_model.predict(X_test)))

Final Accuracy: 0.84


In [93]:
#random forest
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_jobs=2, random_state=14)
clf.fit(X_train, y_train)
print ("Final Accuracy: %s" % accuracy_score(y_test, clf.predict(X_test)))

Final Accuracy: 0.86


In [94]:
#decision trees
from sklearn.tree import DecisionTreeClassifier
clf=DecisionTreeClassifier(max_depth = 4)
clf.fit(X_train, y_train)
print ("Final Accuracy: %s" % accuracy_score(y_test, clf.predict(X_test)))

Final Accuracy: 0.68


In [95]:
#KNN classifier
from sklearn.neighbors import KNeighborsClassifier 
knn = KNeighborsClassifier(n_neighbors = 4).fit(X_train, y_train)
print ("Final Accuracy: %s" % accuracy_score(y_test, knn.predict(X_test)))

Final Accuracy: 0.44


In [96]:
#On using count vectorisation to encode text, it was noticed that
#Random forests give the maximum accuracy when training data is less 
#logistic regression, Multinomial Naive Bayes gives the maximum accuracy when training data is more